# Знакомство с библиотекой `transformers` и `gradio`

Классы `AutoTokenizer` и `AutoModelForCausalLM` позволяет загрузить чекпоинты языковой модели и выполнить генерацию текста.

In [10]:
!pip install gradio

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio

В качестве модели для экспериментов возьмите модель `Qwen/Qwen3-0.6B` с huggingface. Познакомьтесь с описанием модели и её использованием [ссылка](https://huggingface.co/Qwen/Qwen3-0.6B). Веса загружайте в типе `torch.bfloat16` для экономии памяти GPU. Не забудьте в интерфейсе colab подключить GPU.

In [15]:
model_name_or_path = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Входной текст для генерации может быть представлен в режиме диалога в виде списка объектов с полями `role` и `content`. Значения поля `role` может принимать значения `system`, `user`, `assistent`, что соответсвует системному, пользовательскому промптам и ответу модели. У каждой модели специальные токены и собственный формат, приведение к которому происходит с помощью метода `apply_chat_template()`

In [3]:
chat = [
  {"role": "system", "content": "Отвечай на русском языке"},
  {"role": "user", "content": "Что ты думаешь о законах робототехники?"},
  {"role": "assistant", "content": "Законы робототехники — это нарастающая область правовой регуляции, которая стремится адаптироваться к быстрому развитию технологий в области искусственного интеллекта (ИИ) и автоматизации."},
  {"role": "user", "content": "А какой второй закон?"}
]
text = tokenizer.apply_chat_template(chat, tokenize=False,
                                       add_generation_prompt=True,
                                       return_tensors="pt"
)
print(text)

<|im_start|>system
Отвечай на русском языке<|im_end|>
<|im_start|>user
Что ты думаешь о законах робототехники?<|im_end|>
<|im_start|>assistant
Законы робототехники — это нарастающая область правовой регуляции, которая стремится адаптироваться к быстрому развитию технологий в области искусственного интеллекта (ИИ) и автоматизации.<|im_end|>
<|im_start|>user
А какой второй закон?<|im_end|>
<|im_start|>assistant



Генерация выполняется с помощью метода generate(). Генерируемый текст большой (с рассуждениями), поэтому раскройте поле вывода.

In [4]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

In [5]:
all_generated_text = tokenizer.decode(generated_ids[0])
print(all_generated_text)

<|im_start|>system
Отвечай на русском языке<|im_end|>
<|im_start|>user
Что ты думаешь о законах робототехники?<|im_end|>
<|im_start|>assistant
Законы робототехники — это нарастающая область правовой регуляции, которая стремится адаптироваться к быстрому развитию технологий в области искусственного интеллекта (ИИ) и автоматизации.<|im_end|>
<|im_start|>user
А какой второй закон?<|im_end|>
<|im_start|>assistant
<think>
Хорошо, пользователь спрашивает о втором законах робототехники. Нужно понять, что именно подразумевается под "законами". Поскольку пользователь начал с вопроса о законах робототехники, возможно, он хочет узнать о конкретных правилах, которые регулируют использование роботов.

Пользователь уже задал первый вопрос о законах робототехники, и теперь спрашивает о втором. Надо уточнить, что именно подразумевается под "законами". Возможно, он подразумевает правила, регулирующие безопасность, этические аспекты или применение роботов в различных сферах.

Возможно, стоит подчеркнуть

Изучите формат ответа модели и распарсите ответ, например следующим образом.

In [6]:
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
print(f"Рассуждения: {thinking_content}")
print(f"Ответ: {content}")

Рассуждения: <think>
Хорошо, пользователь спрашивает о втором законах робототехники. Нужно понять, что именно подразумевается под "законами". Поскольку пользователь начал с вопроса о законах робототехники, возможно, он хочет узнать о конкретных правилах, которые регулируют использование роботов.

Пользователь уже задал первый вопрос о законах робототехники, и теперь спрашивает о втором. Надо уточнить, что именно подразумевается под "законами". Возможно, он подразумевает правила, регулирующие безопасность, этические аспекты или применение роботов в различных сферах.

Возможно, стоит подчеркнуть, что законы робототехники включают в себя правила безопасности, этические нормы, регулирование использования ИИ, а также вопросы о последствиях автоматизации. Также важно упомянуть, что законодательство в этой области постоянно развивается, чтобы адаптироваться к новым технологиям.

Нужно быть внимательным и обеспечить, что ответ соответствует его запросу, даже если он не полностью понятен. Следу

Используя библиотеку `gradio` можно добавить графический интерфейс. Например, для чатбота будет полезен компонент `ChatInterface`.

**Задание.** Напишите чатбот с графическим интерфейсом и поддержкой истории при генерации ответа. Глубину истории можно задавать, например, через компонент слайдер в интерфейсе.

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def predict(input, history=[]):
    # tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)

    # generate a response
    history = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    # convert the tokens to text, and then split the responses into lines
    response = tokenizer.decode(history[0]).split("<|endoftext|>")
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  # convert to tuples of list
    return response, history

#creating a gradio interface

import gradio as gr

demo = gr.Interface(fn=predict,
             inputs=["text", "state"],
             outputs=["chatbot", "state"])

demo.launch()

/usr/local/lib/python3.12/dist-packages/gradio/components/base.py:437: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  obj = utils.component_or_layout_class(cls_name)(render=render)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://632097802b2e057f94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
